In [18]:
def connect_zambeze_queue():
    import pika
    connection = pika.BlockingConnection(
        pika.ConnectionParameters('localhost', 5672)
    )
    channel = connection.channel()
    channel.queue_declare('hello')
    return channel

def send_zambeze_message(channel):
    from uuid import uuid4
    import json
    another_act_id = str(uuid4())
    act_id = str(uuid4())
    msg = {
            "name": "ImageMagick",
            "activity_id": act_id,
            "campaign_id": "campaign-uuid",
            "origin_agent_id": "def-uuid",
            "files": ["globus://Users/6o1/file.txt"],
            "command": "convert",
            "activity_status": "CREATED",
            "arguments": [
                "-delay",
                "20",
                "-loop",
                "0",
                "~/tests/campaigns/imagesequence/*.jpg",
                "a.gif",
            ],
            "kwargs": {},
            "depends_on": [another_act_id],
    }
    channel.basic_publish(
        exchange="",
        routing_key="hello",
        body=json.dumps(msg),
    )
    return act_id

## Start Zambeze Flowceptor

In [13]:
from flowcept import ZambezeInterceptor
interceptor = ZambezeInterceptor()

In [14]:
from flowcept import FlowceptConsumerAPI
consumer = FlowceptConsumerAPI(interceptor)
consumer.start()

Flowceptor zambeze starting...
... ok!
Flowcept Consumer starting...
 [*] Waiting for Zambeze messages. To exit press CTRL+C
Ok, we're consuming messages!


## Send Zambeze Message

In [ ]:
channel = connect_zambeze_queue()

In [19]:
act_id = send_zambeze_message(channel)
act_id

I'm an interceptor and I need to intercept this:
	{"name": "ImageMagick", "activity_id": "b69c9055-a0b4-4295-b4bb-bb580c1e5afa", "campaign_id": "campaign-uuid", "origin_agent_id": "def-uuid", "files": ["globus://Users/6o1/file.txt"], "command": "convert", "activity_status": "CREATED", "arguments": ["-delay", "20", "-loop", "0", "~/tests/campaigns/imagesequence/*.jpg", "a.gif"], "kwargs": {}, "depends_on": ["674926a5-7eef-4538-921a-b92352d09fe2"]}
Going to send to Redis an intercepted message:
	{"utc_timestamp": 1675799934.364379, "experiment_id": "campaign-uuid", "task_id": "b69c9055-a0b4-4295-b4bb-bb580c1e5afa", "activity_id": "ImageMagick", "dependencies": ["674926a5-7eef-4538-921a-b92352d09fe2"], "custom_metadata": {"command": "convert"}, "status": "SUBMITTED", "used": {"args": ["-delay", "20", "-loop", "0", "~/tests/campaigns/imagesequence/*.jpg", "a.gif"], "kwargs": {}, "files": ["globus://Users/6o1/file.txt"]}, "plugin_id": "zambeze", "user": "root", "sys_name": "Darwin", "node_n

In [20]:
from flowcept.flowcept_api import TaskQueryAPI
query_api = TaskQueryAPI()

Ok, webserver is ready to receive requests.


In [21]:
_filter = {"task_id": act_id}
query_api.query(_filter)

[{'task_id': 'b69c9055-a0b4-4295-b4bb-bb580c1e5afa',
  'utc_timestamp': 1675799934.364379,
  'experiment_id': 'campaign-uuid',
  'activity_id': 'ImageMagick',
  'dependencies': ['674926a5-7eef-4538-921a-b92352d09fe2'],
  'custom_metadata': {'command': 'convert'},
  'status': 'SUBMITTED',
  'used': {'args': ['-delay',
    '20',
    '-loop',
    '0',
    '~/tests/campaigns/imagesequence/*.jpg',
    'a.gif'],
   'files': ['globus://Users/6o1/file.txt']},
  'plugin_id': 'zambeze',
  'user': 'root',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True}]

An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was rec